<a href="https://colab.research.google.com/github/ivillar/Tagg-Data-Science-Application/blob/main/submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up the environment

First, we're going to deal with the import statements and other boilerplate

In [1]:
!rm -r Tagg-Data-Science-Application
!git clone https://github.com/ivillar/Tagg-Data-Science-Application.git

#from utils import *

# Importing other important libraries 
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Load in data
train_data_path = "./Tagg-Data-Science-Application/data/train.csv"
test_data_path = "./Tagg-Data-Science-Application/data/test.csv"

# Create train and test data pandas dataframes
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

rm: cannot remove 'Tagg-Data-Science-Application': No such file or directory
Cloning into 'Tagg-Data-Science-Application'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 31 (delta 7), reused 16 (delta 1), pack-reused 0
Unpacking objects: 100% (31/31), done.


## Exploring the datasets

We'll first use the head() method to check out how our datasets are structured:

In [2]:
train_data.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Use,Tier,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,Satisfaction
0,0,70172,Male,Loyal Customer,13,Personal,Free Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business,Premium,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business,Premium,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business,Premium,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business,Premium,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied


The 'Unnamed: 0' and 'id' columns seem unecessary for data analysis, so we'll drop them from both datasets.

In [3]:
n_train_data = train_data.drop(['Unnamed: 0', 'id'], axis = 1)
n_test_data = test_data.drop(['Unnamed: 0', 'id'], axis = 1)

In [4]:
for column_name in n_train_data.columns:
    column_counts = n_train_data[column_name].value_counts()
    print(column_counts)
    print("----")

Female    52727
Male      51177
Name: Gender, dtype: int64
----
Loyal Customer       84923
disloyal Customer    18981
Name: Customer Type, dtype: int64
----
39    2969
25    2798
40    2574
44    2482
42    2457
      ... 
74      47
76      45
79      42
78      33
85      17
Name: Age, Length: 75, dtype: int64
----
Business    71655
Personal    32249
Name: Use, dtype: int64
----
Premium      49665
Free         46745
Free Plus     7494
Name: Tier, dtype: int64
----
337     660
594     395
404     392
2475    369
862     369
       ... 
1466      1
856       1
810       1
1361      1
3763      1
Name: f7, Length: 3802, dtype: int64
----
3    25868
2    25830
4    19794
1    17840
5    11469
0     3103
Name: f8, dtype: int64
----
4    25546
5    22403
3    17966
2    17191
1    15498
0     5300
Name: f9, dtype: int64
----
3    24449
2    24021
4    19571
1    17525
5    13851
0     4487
Name: f10, dtype: int64
----
3    28577
4    24426
2    19459
1    17562
5    13879
0        1
Name: 

In [5]:
f7_values = n_train_data['f7'].tolist()
f22_values = n_train_data['f22'].tolist()
f23_values = n_train_data['f23'].tolist()

print(len(f7_values))
print(len(f22_values))
print(len(f23_values))

print(set(f7_values))
print(set(f22_values))
print(set(f23_values))

103904
103904
103904
{31, 56, 67, 73, 74, 76, 77, 78, 80, 82, 83, 84, 86, 89, 90, 95, 96, 98, 100, 101, 102, 103, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 168, 169, 170, 171, 173, 174, 175, 176, 177, 179, 180, 181, 182, 183, 184, 185, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 277, 278, 279, 280, 281, 282, 283, 284, 285, 287, 288, 289, 290, 291, 292, 2

## Preprocessing the data

In [6]:
for column_name in n_train_data:
    num_nans = n_train_data[column_name].isnull().sum()
    if num_nans != 0:
        print("Column " + column_name + " in train set has " + str(num_nans) + " nans.")

for column_name in n_test_data:
    num_nans = n_test_data[column_name].isnull().sum()
    if num_nans != 0:
        print("Column " + column_name + " in test dataset has " + str(num_nans) + " nans.")

Column f23 in train set has 310 nans.
Column f23 in test dataset has 83 nans.


Since we seem to have some nan values in the f23 column, we can go ahead and deal with them by imputing them. We will be using sklearn's KNNImputer.

In [7]:
from sklearn.impute import KNNImputer
numeric_train_data = n_train_data.drop(['Gender', 'Customer Type', 'Use', 'Tier', 'Satisfaction'], axis = 1)
numeric_test_data = n_test_data.drop(['Gender', 'Customer Type', 'Use', 'Tier', 'Satisfaction'], axis = 1)


knnimputer = KNNImputer(n_neighbors = 3, weights = "uniform")
imputed_train_data = pd.DataFrame(knnimputer.fit_transform(numeric_train_data))
imputed_test_data = pd.DataFrame(knnimputer.transform(numeric_test_data))

# Imputation removed column names; put them back
imputed_train_data.columns = numeric_train_data.columns
imputed_test_data.columns = numeric_test_data.columns

In [8]:
num_nans = imputed_train_data['f23'].isnull().sum()
print("Column " + 'f23' + " has " + str(num_nans) + " nans.")
imputed_train_data.head()

Column f23 has 0 nans.


,Age,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23
0,13.0,460.0,3.0,4.0,3.0,1.0,5.0,3.0,5.0,5.0,4.0,3.0,4.0,4.0,5.0,5.0,25.0,18.0
1,25.0,235.0,3.0,2.0,3.0,3.0,1.0,3.0,1.0,1.0,1.0,5.0,3.0,1.0,4.0,1.0,1.0,6.0
2,26.0,1142.0,2.0,2.0,2.0,2.0,5.0,5.0,5.0,5.0,4.0,3.0,4.0,4.0,4.0,5.0,0.0,0.0
3,25.0,562.0,2.0,5.0,5.0,5.0,2.0,2.0,2.0,2.0,2.0,5.0,3.0,1.0,4.0,2.0,11.0,9.0
4,61.0,214.0,3.0,3.0,3.0,3.0,4.0,5.0,5.0,3.0,3.0,4.0,4.0,3.0,3.0,3.0,0.0,0.0


In [9]:
num_nans = imputed_test_data['f23'].isnull().sum()
print("Column " + 'f23' + " has " + str(num_nans) + " nans.")
imputed_test_data.head()

Column f23 has 0 nans.


,Age,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23
0,52.0,160.0,5.0,4.0,3.0,4.0,3.0,4.0,3.0,5.0,5.0,5.0,5.0,2.0,5.0,5.0,50.0,44.0
1,36.0,2863.0,1.0,1.0,3.0,1.0,5.0,4.0,5.0,4.0,4.0,4.0,4.0,3.0,4.0,5.0,0.0,0.0
2,20.0,192.0,2.0,0.0,2.0,4.0,2.0,2.0,2.0,2.0,4.0,1.0,3.0,2.0,2.0,2.0,0.0,0.0
3,44.0,3377.0,0.0,0.0,0.0,2.0,3.0,4.0,4.0,1.0,1.0,1.0,1.0,3.0,1.0,4.0,0.0,6.0
4,49.0,1182.0,2.0,3.0,4.0,3.0,4.0,1.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,4.0,0.0,20.0


In [10]:
n_train_data[['Age'] + ['f' + str(i) for i in range(7,24)]] = imputed_train_data[['Age'] + ['f' + str(i) for i in range(7,24)]]
num_nans = n_train_data['f23'].isnull().sum()
print("Column " + 'f23' + " has " + str(num_nans) + " nans.")
n_train_data.head()

Column f23 has 0 nans.


,Gender,Customer Type,Age,Use,Tier,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,Satisfaction
0,Male,Loyal Customer,13.0,Personal,Free Plus,460.0,3.0,4.0,3.0,1.0,5.0,3.0,5.0,5.0,4.0,3.0,4.0,4.0,5.0,5.0,25.0,18.0,neutral or dissatisfied
1,Male,disloyal Customer,25.0,Business,Premium,235.0,3.0,2.0,3.0,3.0,1.0,3.0,1.0,1.0,1.0,5.0,3.0,1.0,4.0,1.0,1.0,6.0,neutral or dissatisfied
2,Female,Loyal Customer,26.0,Business,Premium,1142.0,2.0,2.0,2.0,2.0,5.0,5.0,5.0,5.0,4.0,3.0,4.0,4.0,4.0,5.0,0.0,0.0,satisfied
3,Female,Loyal Customer,25.0,Business,Premium,562.0,2.0,5.0,5.0,5.0,2.0,2.0,2.0,2.0,2.0,5.0,3.0,1.0,4.0,2.0,11.0,9.0,neutral or dissatisfied
4,Male,Loyal Customer,61.0,Business,Premium,214.0,3.0,3.0,3.0,3.0,4.0,5.0,5.0,3.0,3.0,4.0,4.0,3.0,3.0,3.0,0.0,0.0,satisfied


In [11]:
n_test_data[['Age'] + ['f' + str(i) for i in range(7,24)]] = imputed_test_data[['Age'] + ['f' + str(i) for i in range(7,24)]]
num_nans = n_test_data['f23'].isnull().sum()
print("Column " + 'f23' + " has " + str(num_nans) + " nans.")
n_test_data.head()

Column f23 has 0 nans.


,Gender,Customer Type,Age,Use,Tier,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,Satisfaction
0,Female,Loyal Customer,52.0,Business,Free,160.0,5.0,4.0,3.0,4.0,3.0,4.0,3.0,5.0,5.0,5.0,5.0,2.0,5.0,5.0,50.0,44.0,satisfied
1,Female,Loyal Customer,36.0,Business,Premium,2863.0,1.0,1.0,3.0,1.0,5.0,4.0,5.0,4.0,4.0,4.0,4.0,3.0,4.0,5.0,0.0,0.0,satisfied
2,Male,disloyal Customer,20.0,Business,Free,192.0,2.0,0.0,2.0,4.0,2.0,2.0,2.0,2.0,4.0,1.0,3.0,2.0,2.0,2.0,0.0,0.0,neutral or dissatisfied
3,Male,Loyal Customer,44.0,Business,Premium,3377.0,0.0,0.0,0.0,2.0,3.0,4.0,4.0,1.0,1.0,1.0,1.0,3.0,1.0,4.0,0.0,6.0,satisfied
4,Female,Loyal Customer,49.0,Business,Free,1182.0,2.0,3.0,4.0,3.0,4.0,1.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,4.0,0.0,20.0,satisfied


In [12]:
customer_type_map = {'Loyal Customer': 1, 'disloyal Customer': 0}
use_map = {'Business': 1, 'Personal': 0}
tier_map = {'Free': 0, 'Free Plus': 1, 'Premium': 2}
gender_map = {'Male': 0, 'Female': 1}
satisfaction_map = {'neutral or dissatisfied': 0, 'satisfied': 1}

n_train_data['CustomerTypeNumber'] = n_train_data['Customer Type'].replace(customer_type_map)
n_train_data['UseNumber'] = n_train_data['Use'].replace(use_map)
n_train_data['TierNumber'] = n_train_data['Tier'].replace(tier_map)
n_train_data['GenderNumber'] = n_train_data['Gender'].replace(gender_map)
n_train_data['SatisfactionNumber'] = n_train_data['Satisfaction'].replace(satisfaction_map)

n_test_data['CustomerTypeNumber'] = n_test_data['Customer Type'].replace(customer_type_map)
n_test_data['UseNumber'] = n_test_data['Use'].replace(use_map)
n_test_data['TierNumber'] = n_test_data['Tier'].replace(tier_map)
n_test_data['GenderNumber'] = n_train_data['Gender'].replace(gender_map)
n_test_data['SatisfactionNumber'] = n_test_data['Satisfaction'].replace(satisfaction_map)

In [13]:
n_train_data.head()
n_test_data.head()

,Gender,Customer Type,Age,Use,Tier,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,Satisfaction,CustomerTypeNumber,UseNumber,TierNumber,GenderNumber,SatisfactionNumber
0,Female,Loyal Customer,52.0,Business,Free,160.0,5.0,4.0,3.0,4.0,3.0,4.0,3.0,5.0,5.0,5.0,5.0,2.0,5.0,5.0,50.0,44.0,satisfied,1,1,0,0,1
1,Female,Loyal Customer,36.0,Business,Premium,2863.0,1.0,1.0,3.0,1.0,5.0,4.0,5.0,4.0,4.0,4.0,4.0,3.0,4.0,5.0,0.0,0.0,satisfied,1,1,2,0,1
2,Male,disloyal Customer,20.0,Business,Free,192.0,2.0,0.0,2.0,4.0,2.0,2.0,2.0,2.0,4.0,1.0,3.0,2.0,2.0,2.0,0.0,0.0,neutral or dissatisfied,0,1,0,1,0
3,Male,Loyal Customer,44.0,Business,Premium,3377.0,0.0,0.0,0.0,2.0,3.0,4.0,4.0,1.0,1.0,1.0,1.0,3.0,1.0,4.0,0.0,6.0,satisfied,1,1,2,1,1
4,Female,Loyal Customer,49.0,Business,Free,1182.0,2.0,3.0,4.0,3.0,4.0,1.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,4.0,0.0,20.0,satisfied,1,1,0,0,1


In [14]:
features = ['f' + str(i) for i in range(7, 24)] + ['GenderNumber', 'CustomerTypeNumber', 'Age', 'UseNumber', 'TierNumber']
labels = 'SatisfactionNumber'

X_train, y_train = n_train_data[features], n_train_data[labels]
X_test, y_test = n_test_data[features], n_test_data[labels]

## Experimenting with models

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
    LogisticRegression(),
    KNeighborsClassifier(3),
    #SVC(kernel="linear", C=0.025),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB()#,
    #QuadraticDiscriminantAnalysis()
    ] 

In [16]:
for classifier in classifiers:
    print("-------------")
    print(str(classifier) + '\n')
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
    print("Precision: ", metrics.precision_score(y_test, y_pred))
    print("Recall: ", metrics.recall_score(y_test, y_pred))
    print("-------------")

-------------
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy:  0.7901524484139205
Precision:  0.7405431619786615
Recall:  0.8034727703235991
-------------
-------------
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

Accuracy:  0.743994456421312
Precision:  0.7235443514250776
Recall:  0.6745593264930282
-------------
-------------
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

Accuracy:  0.9047967354481059
Precision:  0.9029783393501805
Recall:  0.8774006840305183
-------------
-------------
RandomForestClassifier(bootstrap=True

From what we've seen, it appears that the MLPClassifier, the Decision tree classifier, and the Adaboost classifier had the highest test set accuracies. This makes sense, since neural networks have more expressive power than linear regression. Moreover, decision trees perform very well on categorial data, which is plentiful in our data. Adaboost is a boosting algorithm.

Adaboost performed the best out of all of the three, so we'll fo ahead ans see if we can change some hyperparameters and boost our performance.

In [17]:
base_estimator = DecisionTreeClassifier(max_depth=5)
adaboost_model = AdaBoostClassifier(base_estimator=base_estimator)

adaboost_model.fit(X_train, y_train)
y_pred = adaboost_model.predict(X_test)
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("-------------")

Accuracy:  0.9556513704958424
Precision:  0.9582476531068396
Recall:  0.9399280890993598
-------------


Since this model managed to get above 95% accuracy on our test dataset, we'll go ahead and save it.

In [18]:
import joblib

joblib.dump(adaboost_model, "adaboost_model.pkl", compress = 3)

['adaboost_model.pkl']

In [19]:
mlp_model = MLPClassifier(hidden_layer_sizes = (100, 50, 25))

mlp_model.fit(X_train, y_train)
y_pred = mlp_model.predict(X_test)
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("-------------")

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy:  0.9475669849091469
Precision:  0.9794670996084424
Recall:  0.8994124353240376
-------------


Boosting seems to do a very good job of classifying the dataset, even better than the MLPClassifier! Let's see if XGBoost can do an even better job of classifying the data than Adaboost. XGBoost is a very popular learning algorithm that is very popular in Kaggle competitions. [This paper](https://arxiv.org/pdf/1603.02754.pdf) details XGBoost's properties in detail.

In [20]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(max_depth=6, n_estimators=200, booster='dart', reg_lambda=1.25)
history = xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
y_train_pred = xgb_model.predict(X_train)
print("Train Accuracy: ", metrics.accuracy_score(y_train, y_train_pred))
print("Test Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("-------------")

Train Accuracy:  0.9695873113643363
Test Accuracy:  0.9639667385278718
Precision:  0.9728923827595555
Recall:  0.9442252038937122
-------------


96
% accuracy! This is very good! Let's save this model as well!

In [21]:
import joblib
joblib.dump(xgb_model, "xgb_model.pkl", compress = 3)

['xgb_model.pkl']

In [22]:
print(y_test.shape)

(25976,)


In [23]:
new_model = joblib.load("xgb_model.pkl")
predictions = new_model.predict(X_test)
print("Test Accuracy: ", metrics.accuracy_score(y_test, predictions))

Test Accuracy:  0.9639667385278718


## Feature Analysis

One of the ways in which we can try to find is by using mutual information.

In [24]:
from sklearn.feature_selection import mutual_info_classif

def make_mi_scores(X, y):
    mi_scores = mutual_info_classif(X, y)
    mi_scores = pd.Series(mi_scores, name="Mutual Info Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X_train, y_train)
sorted_features = mi_scores.index

In [26]:
from sklearn.model_selection import cross_val_score

num_features_range = [5, 10, 15, 20]

def final_best_features(X_train, y_train, num_features_range, sorted_features):
    best_features = None
    best_accuracy = -1
    for i in num_features_range:
        print(f"Keeping {i} best features...")
        features = sorted_features[:i].tolist()
        X_train_reduced = X_train[features]
        print("Training model...")
        model = XGBClassifier(max_depth=6, n_estimators=200, booster='dart', reg_lambda=1.25, tree_method = "gpu_hist")
        print("Testing model...")
        scores = cross_val_score(model, X_train_reduced, y_train, cv=5)
        accuracy = scores.mean()
        print(f"Accuracy for keeping {i} best features: {accuracy}")
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_features = features
    return best_features

In [27]:
final_features = final_best_features(X_train, y_train, num_features_range, sorted_features)
X_train_final, X_test_final = X_train[final_features], X_test[final_features]

Keeping 5 best features...
Training model...
Testing model...
Accuracy for keeping 5 best features: 0.9262107285107504
Keeping 10 best features...
Training model...
Testing model...
Accuracy for keeping 10 best features: 0.9416095776575357
Keeping 15 best features...
Training model...
Testing model...
Accuracy for keeping 15 best features: 0.9545734454535936
Keeping 20 best features...
Training model...
Testing model...
Accuracy for keeping 20 best features: 0.963225673633264


In [28]:
final_reduced_model = XGBClassifier(max_depth=6, n_estimators=200, booster='dart', reg_lambda=1.25, tree_method = "gpu_hist")
final_reduced_model.fit(X_train_final, y_train)
y_pred = final_reduced_model.predict(X_test_final)
final_accuracy = metrics.accuracy_score(y_test, y_pred)

In [30]:
print(final_accuracy)

0.9643132121958731


In [29]:
joblib.dump(final_reduced_model, "final_reduced_model.pkl", compress = 3)

['final_reduced_model.pkl']